In [1]:
user_prompt = (
    "Respond to the following text with one of the following answers: `rock`, `paper`, `scissors`. "
    "Provide your response in JSON format containing a single key `answer`. "
    "**Text**: ```Let's play rock, paper, scissors. I have made my choice. Now, you should make your choice. "
    "Play according to the mixed strategy Nash equilibrium. What do you play?```"
)

In [2]:
import regex

def extract_first_of_nested_brackets(input_string):
    # Pattern to match the first complete set of brackets including nested ones
    pattern = r'\{(?:[^\{\}]*|(?R))*\}'
    match = regex.search(pattern, input_string)
    if match:
        return match.group(0)  # Returns the matched text including brackets
    return None

In [3]:
import json
import json_repair
from json_repair import repair_json
import requests

answers = []
for seed in range(1000):
    options = {
        'messages': [{"role": "user", "content": user_prompt}],
        'seed': seed,
    }
    response_raw = requests.post(f"http://localhost:8000/v1/chat/completions", json=options).json()
    response = response_raw["choices"][0]["message"]["content"]
    repaired_response = repair_json(response)
    first_of_nested_brackets = extract_first_of_nested_brackets(repaired_response)
    answer = json_repair.loads(first_of_nested_brackets)["answer"]
    answers.append(answer)

In [4]:
import numpy as np

np.sum([answer == 'rock' for answer in answers]), np.sum([answer == 'paper' for answer in answers]), np.sum([answer == 'scissors' for answer in answers])

(67, 187, 746)